In [1]:
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print('{0}: called {1}'.format(run_dt, fn.__name__))
        return result
    
    return inner

In [2]:
@logged
def func_1():
    pass

In [3]:
@logged
def func_2():
    pass

In [4]:
func_1()

2019-01-21 22:08:44.336577+00:00: called func_1


In [5]:
func_2()

2019-01-21 22:08:56.729865+00:00: called func_2


In [6]:
def timed(fn):
    from time import perf_counter
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start
        
        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k, v) for (k, v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)
        
        print('{0}({1}) took {2:.6f}s to run.'.format(fn.__name__, args_str, elapsed))
        
        return result
    
    return inner

In [13]:
# stacking decorators -- this is eq to fact = logged(timed(fact))
@logged
@timed
def fact(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n + 1))

In [14]:
fact(5)

fact(5) took 0.000013s to run.
2019-01-21 22:11:59.785511+00:00: called fact


120

In [15]:
def fact(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n + 1))

# explicit
fact = logged(timed(fact))

In [16]:
fact(5)

fact(5) took 0.000010s to run.
2019-01-21 22:14:09.684899+00:00: called fact


120

In [17]:
# note: 'logged' is invoked, starts timer, then invokes 'timed', which returns to 'logged'
# this means order of operations matters while stacking decorators
# i.e. invoking function before logging, to ensure call stack returns in appropriate order
#      or ensuring appropriate authorization before invoking function
#      or logging regardless of whether authorized or not (depends on use case)

In [22]:
# ex: logging before invoking function: might not be a good way!
def dec_1(fn):
    def inner():
        print('Running dec_1 - logged')
        return fn()
    return inner

In [23]:
def dec_2(fn):
    def inner():
        print('Running dec_2 - logged')
        return fn()
    return inner

In [24]:
@dec_1
@dec_2
def my_func():
    print('Running my_func')
    
# equivalent to my_func = dec_1(dec_2(my_func))

In [25]:
my_func()

Running dec_1 - logged
Running dec_2 - logged
Running my_func


In [26]:
# ex: invoking function before logging
def dec_1(fn):
    def inner():
        result = fn()
        print('Running dec_1 - logged')
        return result
    return inner

In [27]:
def dec_2(fn):
    def inner():
        result = fn()
        print('Running dec_2 - logged')
        return result
    return inner

In [28]:
@dec_1
@dec_2
def my_func():
    print('Running my_func')

In [30]:
my_func()

Running my_func
Running dec_2 - logged
Running dec_1 - logged
